In [18]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
import re, unicodedata, datetime
from IPython.display import HTML

from __future__ import print_function

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('max_colwidth', 200)

In [19]:
mineduc_file='data/Mineduc Listado IES Vigentes 05-2016.csv'
mineduc=pd.read_csv(mineduc_file, header=0, sep=',')
mineduc['fecha_reconocimiento']=pd.to_datetime(mineduc['fecha_reconocimiento'])
mineduc['año_reconocimiento']=mineduc['fecha_reconocimiento'].dt.year
mineduc.head()

,tipo,n_registro,nombre,tipo_reconocimiento,documento_reconocimiento,n_documento_reconocimiento,fecha_reconocimiento,regimen_funcionamiento,año_reconocimiento
0,C,1,Universidad Gabriela Mistral,Autorización de Funcionamiento,Certificado,S/N,1981-01-12,Autónoma,1981
1,C,2,Universidad Finis Terrae,Autorización de Funcionamiento,Certificado,S/N,1982-02-01,Autónoma,1982
2,C,3,Universidad Diego Portales,Autorización de Funcionamiento,Certificado,1082,1982-12-21,Autónoma,1982
3,C,4,Universidad Central de Chile,Autorización de Funcionamiento,Certificado,125,1983-02-24,Autónoma,1983
4,C,10,Universidad Bolivariana,Autorización de Funcionamiento,Certificado,584,1988-06-21,Autónoma,1988


In [20]:
for i in range(len(mineduc.index)):
    old=mineduc.loc[i,'nombre']
    new=re.sub(" ?\(\+\+\) ?| ?\(\*.*\) ?","", old)
    print("OLD: ", old)
    print("NEW: ", new)
    mineduc.loc[i,'nombre']=new

OLD:  Universidad Gabriela Mistral
NEW:  Universidad Gabriela Mistral
OLD:  Universidad Finis Terrae
NEW:  Universidad Finis Terrae
OLD:  Universidad Diego Portales
NEW:  Universidad Diego Portales
OLD:  Universidad Central de Chile
NEW:  Universidad Central de Chile
OLD:  Universidad Bolivariana
NEW:  Universidad Bolivariana
OLD:  Universidad Pedro de Valdivia
NEW:  Universidad Pedro de Valdivia
OLD:  Universidad Mayor
NEW:  Universidad Mayor
OLD:  Universidad Academia de Humanismo Cristiano
NEW:  Universidad Academia de Humanismo Cristiano
OLD:  Universidad Santo Tomás
NEW:  Universidad Santo Tomás
OLD:  Universidad La República
NEW:  Universidad La República
OLD:  Universidad Sek
NEW:  Universidad Sek
OLD:  Universidad de Las Américas
NEW:  Universidad de Las Américas
OLD:  Universidad Andrés Bello
NEW:  Universidad Andrés Bello
OLD:  Universidad de Viña del Mar
NEW:  Universidad de Viña del Mar
OLD:  Universidad Adolfo Ibañez
NEW:  Universidad Adolfo Ibañez
OLD:  Universidad Iberoa

In [53]:
mineduc_univ=mineduc.query("tipo=='A' | tipo=='B' | tipo=='C'")
mineduc_univ=mineduc_univ.loc[mineduc_univ["nombre"].str.lower().sort_values(ascending=True).index]
mineduc_univ['nombre']
#['nombre'].sort_values(axis=0, ascending=False)

48                        Pontificia Universidad Católica de Chile
51                   Pontificia Universidad Católica de Valparaíso
7                      Universidad Academia de Humanismo Cristiano
14                                       Universidad Adolfo Ibañez
20                                 Universidad Adventista de Chile
31                                     Universidad Alberto Hurtado
12                                        Universidad Andrés Bello
43                                         Universidad Arturo Prat
52                                    Universidad Austral de Chile
18                                   Universidad Autónoma de Chile
28                                  Universidad Bernardo O'Higgins
4                                          Universidad Bolivariana
23              Universidad Católica Cardenal Raúl Silva Henríquez
55                 Universidad Católica de la Santísima Concepción
56                                  Universidad Católica de Te

In [29]:
fondecyt_2010=pd.read_csv('data/fondecyt regular 2010.csv', header=0, sep=',')
fondecyt_2011=pd.read_csv('data/fondecyt regular 2011.csv', header=0, sep=',')
fondecyt_2012=pd.read_csv('data/fondecyt regular 2012.csv', header=0, sep=',')
fondecyt_2013=pd.read_csv('data/fondecyt regular 2013.csv', header=0, sep=',')
fondecyt_2014=pd.read_csv('data/fondecyt regular 2014.csv', header=0, sep=',')
fondecyt_2015=pd.read_csv('data/fondecyt regular 2015.csv', header=0, sep=',')
fondecyt_2016=pd.read_csv('data/fondecyt regular 2016.csv', header=0, sep=',')

In [30]:
fondecyt_regular=pd.concat([fondecyt_2010,fondecyt_2011,fondecyt_2012,fondecyt_2013,fondecyt_2014,fondecyt_2015,fondecyt_2016]).reset_index(drop=True)
for i in range(len(fondecyt_regular.index)):
    name=(fondecyt_regular.loc[i,'nombre']).upper()
    name=name.replace("UNIV.","UNIVERSIDAD").replace("PONT.","PONTIFICIA").replace("CS.","CIENCIAS").replace("TEC.","TECNOLOGIA")
    name=name.replace("INTERNACIONAL SEK","SEK").replace("UNIVERSIDAD CIENCIAS DE LA INFORMATICA","UNIVERSIDAD UCINF")
    fondecyt_regular.loc[i,'nombre']=name.capitalize()

fondecyt_regular.head()

,nombre,n_concursados,n_aprobados,año
0,Universidad de chile,204,86,2010
1,Pontificia universidad catolica de chile,180,100,2010
2,Universidad de concepcion,95,41,2010
3,Universidad austral de chile,48,25,2010
4,Universidad de santiago de chile,62,21,2010


In [31]:
for i in range(len(fondecyt_regular.index)):
    if fondecyt_regular.ix[i,'nombre'] != 'Otras universidades':
        d = mineduc.apply(lambda x: fuzz.ratio(x['nombre'].upper(), fondecyt_regular.ix[i, 'nombre'].upper()), axis=1)
        if d.max()<85:
            print("WARNING - Fuzz ratio lower than 85")
            print('Fondecyt name: ', fondecyt_regular.ix[i,'nombre'])
            print('Mineduc name: ', mineduc.ix[d.idxmax(), 'nombre'])
        fondecyt_regular.ix[i,'nombre']=mineduc.ix[d.idxmax(), 'nombre']

fondecyt_regular.head()

WARNING - Fuzz ratio lower than 85
Fondecyt name:  Universidad del bío-bío
Mineduc name:  Universidad del Bío-Bío


,nombre,n_concursados,n_aprobados,año
0,Universidad de Chile,204,86,2010
1,Pontificia Universidad Católica de Chile,180,100,2010
2,Universidad de Concepción,95,41,2010
3,Universidad Austral de Chile,48,25,2010
4,Universidad de Santiago de Chile,62,21,2010


In [24]:
fondecyt_2010=pd.read_csv('data/fondecyt postdoc 2010.csv', header=0, sep=',')
fondecyt_2011=pd.read_csv('data/fondecyt postdoc 2011.csv', header=0, sep=',')
fondecyt_2012=pd.read_csv('data/fondecyt postdoc 2012.csv', header=0, sep=',')
fondecyt_2013=pd.read_csv('data/fondecyt postdoc 2013.csv', header=0, sep=',')
fondecyt_2014=pd.read_csv('data/fondecyt postdoc 2014.csv', header=0, sep=',')
fondecyt_2015=pd.read_csv('data/fondecyt postdoc 2015.csv', header=0, sep=',')
fondecyt_2016=pd.read_csv('data/fondecyt postdoc 2016.csv', header=0, sep=',')

In [27]:
fondecyt_postdoc=pd.concat([fondecyt_2010,fondecyt_2011,fondecyt_2012,fondecyt_2013,fondecyt_2014,fondecyt_2015,fondecyt_2016]).reset_index(drop=True)
for i in range(len(fondecyt_postdoc.index)):
    name=(fondecyt_postdoc.loc[i,'nombre']).upper()
    name=name.replace("UNIV.","UNIVERSIDAD").replace("PONT.","PONTIFICIA").replace("CS.","CIENCIAS").replace("TEC.","TECNOLOGIA")
    name=name.replace("INTERNACIONAL SEK","SEK").replace("UNIVERSIDAD CIENCIAS DE LA INFORMATICA","UNIVERSIDAD UCINF")
    fondecyt_postdoc.loc[i,'nombre']=name.capitalize()

fondecyt_postdoc.head()

,nombre,n_concursados,n_aprobados,año
0,Universidadde chile,35,23,2010
1,Pontificia universidad catolica de chile,26,16,2010
2,Universidadde concepcion,13,8,2010
3,Universidadtecnica federico santa maria,8,5,2010
4,Universidadde talca,6,4,2010


In [28]:
for i in range(len(fondecyt_postdoc.index)):
    if fondecyt_postdoc.ix[i,'nombre'] != 'Otras universidades':
        d = mineduc.apply(lambda x: fuzz.ratio(x['nombre'].upper(), fondecyt_postdoc.ix[i, 'nombre'].upper()), axis=1)
        if d.max()<85:
            print("WARNING - Fuzz ratio lower than 85")
            print('Fondecyt name: ', fondecyt_postdoc.ix[i,'nombre'])
            print('Mineduc name: ', mineduc.ix[d.idxmax(), 'nombre'])
        fondecyt_postdoc.ix[i,'nombre']=mineduc.ix[d.idxmax(), 'nombre']

fondecyt_postdoc.head()

WARNING - Fuzz ratio lower than 85
Fondecyt name:  Universidad del bío-bío
Mineduc name:  Universidad del Bío-Bío
WARNING - Fuzz ratio lower than 85
Fondecyt name:  Universidad del bío-bío
Mineduc name:  Universidad del Bío-Bío
WARNING - Fuzz ratio lower than 85
Fondecyt name:  Universidad católica de la stma. concepción
Mineduc name:  Universidad Católica de la Santísima Concepción


,nombre,n_concursados,n_aprobados,año
0,Universidad de Chile,35,23,2010
1,Pontificia Universidad Católica de Chile,26,16,2010
2,Universidad de Concepción,13,8,2010
3,Universidad Técnica Federico Santa María,8,5,2010
4,Universidad de Talca,6,4,2010


In [33]:
pd.concat([fondecyt_regular['nombre'], fondecyt_postdoc['nombre']]).drop_duplicates().sort_values()

30                                             Otras universidades
1                         Pontificia Universidad Católica de Chile
6                    Pontificia Universidad Católica de Valparaíso
22                     Universidad Academia de Humanismo Cristiano
18                                       Universidad Adolfo Ibañez
240                                Universidad Adventista de Chile
20                                     Universidad Alberto Hurtado
8                                         Universidad Andrés Bello
21                                         Universidad Arturo Prat
3                                     Universidad Austral de Chile
140                                  Universidad Autónoma de Chile
141                                 Universidad Bernardo O'Higgins
64                                         Universidad Bolivariana
26              Universidad Católica Cardenal Raúl Silva Henríquez
27                                  Universidad Católica de Te